# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. 
Дополнительно нужно измеряйть *AUC-ROC* и сравнивайть её значение с *F1*-мерой.


# 1. Подготовка данных

In [1]:
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier


На данном этапе я добавляю необходимые библиотеки для работы

In [2]:
data = pd.read_csv('/datasets/Churn.csv')
print(data.shape)

#data.head(10)

(10000, 14)


In [3]:
print(data['Tenure'].mean())

4.997690023099769


In [4]:
data['Tenure'] = data['Tenure'].fillna(data['Tenure'].mean())
data.isna().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

Теперь я сделал замену на среднее пустых значений, чтобы этот параматер координально не изменился

Здесь я делаю вычитку файла, а также заполняю пропуски в столбце "Tenure"

In [5]:

data_ohe = pd.get_dummies(data, drop_first=True)


Преобразую данные методом прямого кодирования

In [6]:
df_1, df_test = train_test_split(data_ohe, test_size = 0.2, random_state=12345)

features_test = df_test.drop(['Exited'], axis=1)
target_test = df_test['Exited']

numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'EstimatedSalary']

print(features_test.shape)

(2000, 2944)


In [7]:
df_train, df_valid = train_test_split(df_1, test_size=0.2, random_state=12345)

features_train = df_train.drop(['Exited'], axis=1)
target_train = df_train['Exited']
features_valid = df_valid.drop(['Exited'], axis=1)
target_valid = df_valid['Exited']

scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

print(features_train.shape)
print(features_valid.shape)


(6400, 2944)
(1600, 2944)


Здесь я разделил данные на тестовую, тренировочную, валидационную выборки. Разделил их на признаки и целевой признак. 

# 2. Исследование задачи

In [8]:
model = LogisticRegression(random_state=12345, solver='liblinear', class_weight = 'balanced')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print("F1:", f1_score(target_valid, predicted_valid))

F1: 0.45464982778415614


In [9]:
for i in range(10, 100, 10):
    model = RandomForestClassifier(random_state=12345, n_estimators = i)
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    print("F1:", f1_score(target_valid, predicted_valid), ' , n_estimators =',i)

F1: 0.4272300469483568  , n_estimators = 10
F1: 0.4551724137931034  , n_estimators = 20
F1: 0.48526077097505665  , n_estimators = 30
F1: 0.490990990990991  , n_estimators = 40
F1: 0.5011286681715575  , n_estimators = 50
F1: 0.502283105022831  , n_estimators = 60
F1: 0.5056433408577878  , n_estimators = 70
F1: 0.5011286681715575  , n_estimators = 80
F1: 0.5022421524663677  , n_estimators = 90


Здесь я обучал модели и замерял метрику f1 на несбалансированных данных

# 3. Борьба с дисбалансом

In [10]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled_train, target_upsampled_train = upsample(features_train, target_train, 3)

print(features_upsampled_train.shape)
print(target_upsampled_train.shape)

(9014, 2944)
(9014,)


На данном этапеи я увеличил выборку в 3 раза и разделил ее на положительные и отрицательные обьекты

In [11]:
model_upsample_lr = LogisticRegression(random_state=12345, solver='liblinear', class_weight = 'balanced')
model_upsample_lr.fit(features_upsampled_train, target_upsampled_train)
predicted_valid_lr = model_upsample_lr.predict(features_valid)

print("F1:", f1_score(target_valid, predicted_valid_lr))

F1: 0.46448703494926724


Здесь я замерил f1 на модели логистической регрессии

In [12]:
for i in range(10, 120, 10):
    model_rand = RandomForestClassifier(random_state=12345, n_estimators = i)
    model_rand.fit(features_upsampled_train, target_upsampled_train)
    predicted_valid_rand = model_rand.predict(features_valid)
    probabilities_valid = model_rand.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc = roc_auc_score(target_valid ,probabilities_one_valid)
    
    print("F1:", f1_score(target_valid, predicted_valid_rand), ' , n_estimators = ', i)
    print("AUC-ROC",auc_roc)

F1: 0.49173553719008256  , n_estimators =  10
AUC-ROC 0.776395642648305
F1: 0.5247933884297521  , n_estimators =  20
AUC-ROC 0.8045425976676311
F1: 0.5542168674698795  , n_estimators =  30
AUC-ROC 0.8200569478690352
F1: 0.5588822355289421  , n_estimators =  40
AUC-ROC 0.8209488258000819
F1: 0.5537848605577689  , n_estimators =  50
AUC-ROC 0.825371318936057
F1: 0.5657370517928287  , n_estimators =  60
AUC-ROC 0.8307760228605745
F1: 0.5725646123260437  , n_estimators =  70
AUC-ROC 0.8318307543938666
F1: 0.5800000000000001  , n_estimators =  80
AUC-ROC 0.834211216032937
F1: 0.5720000000000001  , n_estimators =  90
AUC-ROC 0.8367443529241128
F1: 0.5765407554671969  , n_estimators =  100
AUC-ROC 0.8376540429679051
F1: 0.5628742514970061  , n_estimators =  110
AUC-ROC 0.8371591206923312


In [13]:
model_rand = RandomForestClassifier(random_state=12345, n_estimators = 80)
model_rand.fit(features_upsampled_train, target_upsampled_train)
predicted_valid_rand = model_rand.predict(features_valid)
probabilities_valid = model_rand.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid ,probabilities_one_valid)

print("AUC-ROC",auc_roc)
print("F1:", f1_score(target_valid, predicted_valid_rand))

AUC-ROC 0.834211216032937
F1: 0.5800000000000001


После я обучил модель случайный лес и нашел лучший результат f1 при разных параметрах. И получил результат, который стал лучше, чем до балансипрования классов

In [14]:
for i in range(1, 10):
    model_tr = DecisionTreeClassifier(random_state = 12345, max_depth = i)
    model_tr.fit(features_upsampled_train, target_upsampled_train)
    predicted_valid_tr = model_tr.predict(features_valid)
    print("F1:", f1_score(target_valid, predicted_valid_tr), ' ,max_depth = ',i)

F1: 0.47058823529411764  ,max_depth =  1
F1: 0.5006075334143378  ,max_depth =  2
F1: 0.5330948121645795  ,max_depth =  3
F1: 0.5283474065138722  ,max_depth =  4
F1: 0.563165905631659  ,max_depth =  5
F1: 0.5703124999999999  ,max_depth =  6
F1: 0.5667125171939477  ,max_depth =  7
F1: 0.5501432664756446  ,max_depth =  8
F1: 0.5279583875162548  ,max_depth =  9


Наилучший результат получился у модели случайного леса при увеличении выборки

In [15]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled_train, target_downsampled_train = downsample(features_train, target_train, 0.1)
print(features_downsampled_train.shape)
print(target_downsampled_train.shape)

(1816, 2944)
(1816,)


На данном этапе я уменьшаю выборку и буду обучать модели на основе данной выборки

In [16]:
for i in range(10, 120, 10):
    model_rand_1 = RandomForestClassifier(random_state=12345, n_estimators = i)
    model_rand_1.fit(features_downsampled_train, target_downsampled_train)
    predicted_valid_rand = model_rand_1.predict(features_valid)
    probabilities_valid = model_rand_1.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc = roc_auc_score(target_valid ,probabilities_one_valid)
    
    print("F1:", f1_score(target_valid, predicted_valid_rand), ' , n_estimators = ', i)
    print("AUC-ROC",auc_roc)
    print()

F1: 0.39648866130212146  , n_estimators =  10
AUC-ROC 0.7450157891656553

F1: 0.38828337874659397  , n_estimators =  20
AUC-ROC 0.777858780480978

F1: 0.3870967741935484  , n_estimators =  30
AUC-ROC 0.7848767529027383

F1: 0.3881401617250674  , n_estimators =  40
AUC-ROC 0.8018122552424864

F1: 0.38866396761133604  , n_estimators =  50
AUC-ROC 0.8061469601085012

F1: 0.38933333333333336  , n_estimators =  60
AUC-ROC 0.8102920932031522

F1: 0.3847167325428195  , n_estimators =  70
AUC-ROC 0.8134651938594014

F1: 0.3884134298880843  , n_estimators =  80
AUC-ROC 0.8167833360051502

F1: 0.38562091503267976  , n_estimators =  90
AUC-ROC 0.8188698977839186

F1: 0.3841145833333333  , n_estimators =  100
AUC-ROC 0.8206116679516834

F1: 0.3847150259067358  , n_estimators =  110
AUC-ROC 0.8199259016109783



In [17]:
model_downsample_lr = LogisticRegression(random_state=12345, solver='liblinear', class_weight = 'balanced')
model_downsample_lr.fit(features_downsampled_train, target_downsampled_train)
predicted_valid_lr = model_downsample_lr.predict(features_valid)

print("F1:", f1_score(target_valid, predicted_valid_lr))

F1: 0.4594894561598224


In [18]:
for i in range(1, 10):
    model_tr = DecisionTreeClassifier(random_state = 12345, max_depth = i)
    model_tr.fit(features_downsampled_train, target_downsampled_train)
    predicted_valid_tr = model_tr.predict(features_valid)
    print("F1:", f1_score(target_valid, predicted_valid_tr), ' ,max_depth = ',i)

F1: 0.3184445612191277  ,max_depth =  1
F1: 0.3184445612191277  ,max_depth =  2
F1: 0.4164827078734364  ,max_depth =  3
F1: 0.4167893961708394  ,max_depth =  4
F1: 0.44573322286661143  ,max_depth =  5
F1: 0.4337539432176657  ,max_depth =  6
F1: 0.42576590730557745  ,max_depth =  7
F1: 0.4224207961007311  ,max_depth =  8
F1: 0.42704039571310803  ,max_depth =  9


На основе показателей, можно сказать, что лучший результат получился при увеличении выборки и при использовании модели случайного леса.

# 4. Тестирование модели

In [19]:
predicted_test = model_rand.predict(features_test)
f1 = f1_score(target_test, predicted_test)
print("F1:", f1)
probabilities_test = model_rand.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
auc_roc = roc_auc_score(target_test ,probabilities_one_test)
print("AUC-ROC",auc_roc)

F1: 0.5408618127786032
AUC-ROC 0.8430384816375874


На тестовой выборке я получил меру f1 примерно равную 0.54 и значение параметра AUC-ROC = 0.86. Это говорит о том, что после обработки данных модель случайный лес по показаниям метрик стал лучше предсказывать, чем это было до.